# Final Project Robotics Manipulation

## Contents

The following contents of the assignment are:

1. code/ 
    - Directory of the code files
        - `mile2.py`: Contains the TrajectoryGenerator function used to craft the trajectory for milestone2
        - `run.py`: Main executable for milestone 2
2. output.csv
    - Example csv used to produce coppelia sim animation with scene 8
3. KingShepard_Ryan_milestone2.mp4 
    - Video of Coppelia Sim Animation 
4. README.pdf
    - TIS I, THE DOCUMENT BEFORE YOU!

## Code Dependencies

The code for milestone2 depends on the modern_robotics package, numpy, argparse and csv. The Python Standard Libraries should already contain argparse and csv; if they are not present, use `pip install [package]` to add them to your python libs. You will need to install modern_robotics(instructions [here](https://github.com/NxRLab/ModernRobotics/tree/master/packages/Python)) and numpy(instructions [here](https://numpy.org/install/))

## Code Execution

The main execution script for this milestone is `rain.py`. This file interfaces with the user for arguments and calls the TrajectoryGenerator in `mile2.py` to produce the outputs

```bash
./run.py -h
usage: run.py [-h] [--ani-file ANI_FILE] [--only_mile1] [--only_mile2]
              [--only_mile3]

Calculate and output csv for final project animations

optional arguments:
  -h, --help           show this help message and exit
  --ani-file ANI_FILE  output csv for animations. Currently only supports
                       milestone 2. Defaults to "ani_file.csv"
  --only_mile1         produce only the outputs for milestone 1
  --only_mile2         produce only the outputs for milestone 2
  --only_mile3         produce only the outputs for milestone 3

```

To produce a file called `output.csv` with the transformation matrices and gripper commands run: 

`python3 run.py --only_mile2 --ani-file 'output.csv'`

